## MVTEc    Visualizer

- claculates clip embeddings
- vizualisation via tensorboard projector

In [1]:
# activate clip env
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt
import numpy as np
import os
import clip
import torch
import pandas as pd
import seaborn as sns
from sklearn.manifold import TSNE


import warnings
warnings.filterwarnings("ignore", message=".*pandas only supports SQLAlchemy connectable.*")


torch.cuda.empty_cache()
print(f'GPU is available: {torch.cuda.is_available()}')
print("Torch version:", torch.__version__)


from PIL import Image

import torchvision
import torch.utils.tensorboard 
from torch.utils.tensorboard import SummaryWriter


GPU is available: True
Torch version: 2.1.0+cu118


2023-11-12 16:10:41.678618: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-11-12 16:10:41.696910: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-12 16:10:41.696928: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-12 16:10:41.696942: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-12 16:10:41.700697: I tensorflow/core/platform/cpu_feature_g

In [2]:

def convert_images_for_tensorboard(image_paths):
    # Preprocess the images
    transform_display = torchvision.transforms.Compose([
        torchvision.transforms.Resize(64),  # Resize the images 
        torchvision.transforms.CenterCrop(64),  # Crop the images
        torchvision.transforms.ToTensor()  # Convert images to tensors        
    ])

    images = []  # Array to store the preprocessed images
    for path in image_paths:
        image = Image.open(path).convert('RGB')  # Open the image and convert to RGB
        # Apply the display transformation
        image_display = transform_display(image)
        image_display = image_display.unsqueeze(0) # Add a batch dimension to the image
        images.append(image_display)

    images = torch.cat(images, dim=0)  # Concatenate the images along the batch dimension

    return images

In [6]:
# PATHs
PROJECT_DATA_PATH='//home/bule/projects/MVTec_Visualizer/data/mvtec_anomaly_detection'
LOG_DIR= '/home/bule/projects/MVTec_Visualizer/tensorboard_logs'

In [7]:
# paths to images
image_paths = []
for root, dirs, files in os.walk(PROJECT_DATA_PATH):
    if 'ground_truth' not in root:
        for file in files:
            if file.endswith('.png'):
                image_paths.append(os.path.join(root, file))
len(image_paths)






5354

In [8]:
BATCH_SIZE = 1024  # Adjust this based on your GPU memory
FILENAME = 'MVTEC_CLIP_embeddings_df'

if not os.path.exists(os.path.join(PROJECT_DATA_PATH,FILENAME+'.pkl')):

    # Load the model
    device = "cuda" if torch.cuda.is_available() else "cpu"
    model, transform = clip.load('ViT-B/32', device=device)


    # Function to get embeddings for a batch of images
    def get_batch_embeddings(batch_paths):
        images = [Image.open(p) for p in batch_paths]
        tensors = [transform(img) for img in images]
        batch_tensor = torch.stack(tensors).to(device)
        with torch.no_grad():
            batch_features = model.encode_image(batch_tensor)
        return batch_features.cpu().numpy()

    # Calculate embeddings for all images in batches
    all_embeddings = []
    for i in range(0, len(image_paths), BATCH_SIZE):
        batch_paths = image_paths[i:i+BATCH_SIZE]
        batch_embeddings = get_batch_embeddings(batch_paths)
        all_embeddings.extend(batch_embeddings)
        print(f"Processed images {i} to {i + len(batch_embeddings)}")

    # Convert all embeddings to DataFrame and save
    df = pd.DataFrame(all_embeddings, index=image_paths)
    df.to_pickle(os.path.join(PROJECT_DATA_PATH, FILENAME + '.pkl'))
else:
    df = pd.read_pickle(os.path.join(PROJECT_DATA_PATH,FILENAME+'.pkl'))

df.head()


Processed images 0 to 1024
Processed images 1024 to 2048
Processed images 2048 to 3072
Processed images 3072 to 4096
Processed images 4096 to 5120
Processed images 5120 to 5354


,0,1,2,3,4,5,6,7,8,9,...,502,503,504,505,506,507,508,509,510,511
//home/bule/projects/MVTec_Visualizer/data/mvtec_anomaly_detection/transistor/test/damaged_case/001.png,0.186768,-0.099304,-0.444092,-0.162842,0.243652,-0.088989,0.325928,0.600098,-0.354004,0.196777,...,-0.354980,-0.232910,0.734863,-0.283691,0.000609,-0.693848,0.053375,0.746094,0.341797,-0.132080
//home/bule/projects/MVTec_Visualizer/data/mvtec_anomaly_detection/transistor/test/damaged_case/009.png,0.204224,0.108948,-0.504395,-0.085022,0.519531,-0.298340,0.139526,0.593750,-0.199219,0.260742,...,-0.326660,-0.277344,0.656250,-0.329834,-0.215210,-0.642090,-0.032318,0.791016,0.173584,-0.114746
//home/bule/projects/MVTec_Visualizer/data/mvtec_anomaly_detection/transistor/test/damaged_case/007.png,0.012260,0.116394,-0.501953,-0.183838,0.270020,-0.191895,0.183105,0.643555,-0.268066,0.210327,...,-0.252930,-0.449219,0.693359,-0.274658,-0.049103,-0.625977,-0.104919,0.709473,0.172241,-0.039185
//home/bule/projects/MVTec_Visualizer/data/mvtec_anomaly_detection/transistor/test/damaged_case/003.png,0.122864,-0.008751,-0.594238,-0.012032,0.351562,-0.247681,0.293945,0.671875,-0.291016,-0.003828,...,-0.080322,-0.366455,0.833008,-0.174805,0.009460,-0.578613,0.047913,0.584961,0.180420,-0.087341
//home/bule/projects/MVTec_Visualizer/data/mvtec_anomaly_detection/transistor/test/damaged_case/000.png,0.033691,0.095276,-0.407227,-0.280029,0.268311,-0.163940,0.309814,0.631836,-0.463379,0.166260,...,-0.287598,-0.335693,0.934082,-0.235352,-0.063538,-0.691406,-0.154785,0.882812,0.130737,-0.209106


In [12]:
# make tensorboard logs

print(FILENAME)
TENSORBOARD_LOGS_PATH=os.path.join(LOG_DIR,'_'+FILENAME).replace(" ", "_")


if not  os.path.exists(TENSORBOARD_LOGS_PATH):

    df = pd.read_pickle(os.path.join(PROJECT_DATA_PATH,FILENAME+'.pkl'))

    # Convert the resulting series of lists to a NumPy array
    numpy_array = df.to_numpy()

    # resize images 
    images = convert_images_for_tensorboard(image_paths)

    # for tensorboard
    writer = SummaryWriter(TENSORBOARD_LOGS_PATH)
    writer.add_embedding(numpy_array, label_img=images)

else:
    df = pd.read_pickle(os.path.join(PROJECT_DATA_PATH,FILENAME+'.pkl'))

print(f'{FILENAME}  number of images:{len(image_paths)}')

MVTEC_embeddings_df
MVTEC_embeddings_df  number of images:5354


### run tensorboard on port forward to browser

In [13]:
!tensorboard  --logdir $TENSORBOARD_LOGS_PATH

2023-11-12 16:21:33.977256: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-11-12 16:21:33.978493: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-12 16:21:33.995517: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-12 16:21:33.995536: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-12 16:21:33.995548: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to regi